## Experiments for binary treatment effect estimation comparison

In [1]:
import sys, os

# add the project root to sys.path
root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root not in sys.path:
    sys.path.insert(0, root)

from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')

import CausalEGM as cegm
# import the module
from models import *

import numpy as np
import pickle
import os
from tqdm import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)



np.random.seed(42)
n_tr = 1000
n_p = 1000

nI = 2
nX = 2
nO = 2
nS= 2
p = nI+nX+nO+nS
ate = 2
beta_cov = 0
strength_instr = 1
strength_conf = 1
strength_outcome = 1
binary_intervention=True

## Example of hyperparameter tuning

In [10]:
# def make_synthetic_data(N=1000, D=5, seed=0):
#     torch.manual_seed(seed)
#     X = torch.randn(N, D)
#     w_p, b_p = torch.randn(D), 0.1
#     p = torch.sigmoid(X @ w_p + b_p)
#     t = torch.bernoulli(p)
#     beta0 = torch.randn(D)
#     y0 = X @ beta0 + 0.1 * torch.randn(N)
#     tau = (X[:,0] * 2.0).clamp(min=0)
#     y1 = y0 + tau + 0.1 * torch.randn(N)
#     y = y0 * (1 - t) + y1 * t
#     return X.numpy(), t.numpy(), y.numpy(), y0.numpy(), y1.numpy()

# X, t, y, y0, y1 = make_synthetic_data(N=400, D=2)
# X_train, X_tmp, t_train, t_tmp, y_train, y_tmp = train_test_split(X, t, y, test_size=0.4, random_state=42)
# X_val, X_test, t_val, t_test, y_val, y_test = train_test_split(X_tmp, t_tmp, y_tmp, test_size=0.5, random_state=42)

def tune_and_eval(model_name, X_train, t_train, y_train, X_val, t_val, y_val, X_test, t_test, y_test, n_trials=20):
    def objective(trial):
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
        wd = trial.suggest_loguniform("wd", 1e-7, 1e-4)
        bs = trial.suggest_categorical("bs", [32, 64, 128])
        epochs = trial.suggest_int("epochs", 50, 200)
        if model_name == "tarnet":
            rep1 = trial.suggest_int("rep1", 50, 200)
            rep2 = trial.suggest_int("rep2", 50, 200)
            head = trial.suggest_int("head", 50, 200)
            drop = trial.suggest_uniform("drop", 0.0, 0.5)
            trainer = TARNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop)
        elif model_name == "cfrnet":
            rep1 = trial.suggest_int("rep1", 50, 200)
            rep2 = trial.suggest_int("rep2", 50, 200)
            head = trial.suggest_int("head", 50, 200)
            drop = trial.suggest_uniform("drop", 0.0, 0.5)
            ipm_w = trial.suggest_loguniform("ipm_weight", 0.01, 10.0)
            trainer = CFRNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop, ipm_w)
        elif model_name == "cevae":
            ld = trial.suggest_int("latent_dim", 10, 200)
            hd = trial.suggest_int("hidden_dim", 20, 400)
            nl = trial.suggest_int("num_layers", 2, 5)
            ns = trial.suggest_categorical("num_samples", [10,50,100,200])
            trainer = CEVAETrainer(X_train.shape[1], ld, hd, nl, ns)
        else:  # dragonnet
            sh = trial.suggest_int("shared_hidden", 50, 200)
            oh = trial.suggest_int("outcome_hidden", 50, 200)
            trainer = DragonNetTrainer(X_train.shape[1], sh, oh)
        return trainer.fit(X_train, t_train, y_train, X_val, t_val, y_val,
                            lr=lr, weight_decay=wd, batch_size=bs, epochs=epochs)
    
    study = optuna.create_study(direction="minimize", study_name=f"{model_name}_tune")
    study.optimize(objective, n_trials=n_trials)
    best = study.best_params
    print(f"Best params for {model_name}: {best}")
    # retrain
    X_trn = np.vstack([X_train, X_val])
    t_trn = np.concatenate([t_train, t_val])
    y_trn = np.concatenate([y_train, y_val])
    if model_name == "tarnet":
        trainer = TARNetTrainer(X_trn.shape[1], [best['rep1'],best['rep2']], [best['head']], best['drop'])
    elif model_name == "cfrnet":
        trainer = CFRNetTrainer(X_trn.shape[1], [best['rep1'],best['rep2']], [best['head']], best['drop'], best['ipm_weight'])
    elif model_name == "cevae":
        trainer = CEVAETrainer(X_trn.shape[1], best['latent_dim'], best['hidden_dim'], best['num_layers'], best['num_samples'])
    else:
        trainer = DragonNetTrainer(X_trn.shape[1], best['shared_hidden'], best['outcome_hidden'])
    trainer.fit(X_trn, t_trn, y_trn, X_test, t_test, y_test,
                lr=best['lr'], weight_decay=best['wd'], batch_size=best['bs'], epochs=best['epochs'])
    if model_name == "cevae":
        return trainer.predict(X_test)
    else:
        y0p, y1p = trainer.predict(X_test)
        return y1p - y0p

# # Tune and train each model separately, storing ITEs
# print("Tuning and training TARNet...")
# ite_tarnet = tune_and_eval("tarnet")
# print("TARNet ITE shape:", ite_tarnet.shape)

# print("Tuning and training CFRNet...")
# ite_cfrnet = tune_and_eval("cfrnet")
# print("CFRNet ITE shape:", ite_cfrnet.shape)

# print("Tuning and training CEVAE...")
# ite_cevae = tune_and_eval("cevae")
# print("CEVAE ITE shape:", ite_cevae.shape)

# print("Tuning and training DragonNet...")
# ite_dragonnet = tune_and_eval("dragonnet")
# print("DragonNet ITE shape:", ite_dragonnet.shape)

# # Now you have separate ITE arrays:
# # ite_tarnet, ite_cfrnet, ite_cevae, ite_dragonnet
# print("All models complete.")


In [21]:
from functools import lru_cache

def tune_and_eval(model_name,
                  X_train, t_train, y_train,
                  X_val,   t_val,   y_val,
                  X_test,  t_test,  y_test,
                  provided_params=None,
                  n_trials=20):
    """
    If best_params is None: runs Optuna, returns (ITE_array, best_params).
    If best_params is given: skips Optuna, returns ITE_array only.
    """
    # 1) hyperparam search
    if provided_params is None:
        import optuna
        study = optuna.create_study(direction="minimize",
                                    study_name=f"{model_name}_tune")
        def objective(trial):
            # common
            lr     = trial.suggest_loguniform("lr", 1e-5, 1e-2)
            wd     = trial.suggest_loguniform("wd", 1e-7, 1e-4)
            bs     = trial.suggest_categorical("bs", [32, 64, 128])
            epochs = trial.suggest_int("epochs", 50, 200)

            # model‐specific
            if model_name == "tarnet":
                rep1 = trial.suggest_int("rep1", 50, 200)
                rep2 = trial.suggest_int("rep2", 50, 200)
                head = trial.suggest_int("head", 50, 200)
                drop = trial.suggest_uniform("drop", 0.0, 0.5)
                trainer = TARNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop)

            elif model_name == "cfrnet":
                rep1   = trial.suggest_int("rep1", 50, 200)
                rep2   = trial.suggest_int("rep2", 50, 200)
                head   = trial.suggest_int("head", 50, 200)
                drop   = trial.suggest_uniform("drop", 0.0, 0.5)
                ipm_w  = trial.suggest_loguniform("ipm_weight", 0.01, 10.0)
                trainer = CFRNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop, ipm_w)

            elif model_name == "cevae":
                ld = trial.suggest_int("latent_dim", 10, 200)
                hd = trial.suggest_int("hidden_dim", 20, 400)
                nl = trial.suggest_int("num_layers", 2, 5)     # note: 2→5 to avoid pop error
                ns = trial.suggest_categorical("num_samples", [10,50,100,200])
                trainer = CEVAETrainer(X_train.shape[1], ld, hd, nl, ns)

            else:  # dragonnet
                sh = trial.suggest_int("shared_hidden", 50, 200)
                oh = trial.suggest_int("outcome_hidden", 50, 200)
                trainer = DragonNetTrainer(X_train.shape[1], sh, oh)

            return trainer.fit(
                X_train, t_train, y_train,
                X_val,   t_val,   y_val,
                lr=lr, weight_decay=wd,
                batch_size=bs, epochs=epochs
            )

        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        print(f"🔍 Best params for {model_name}: {best_params}")
    else:
        best_params = provided_params
    # 2) retrain on combined train+val
    X_trn = np.vstack([X_train, X_val])
    t_trn = np.concatenate([t_train, t_val])
    y_trn = np.concatenate([y_train, y_val])

    if model_name == "tarnet":
        trainer = TARNetTrainer(
            X_trn.shape[1],
            [best_params['rep1'], best_params['rep2']],
            [best_params['head']],
            best_params['drop']
        )
    elif model_name == "cfrnet":
        trainer = CFRNetTrainer(
            X_trn.shape[1],
            [best_params['rep1'], best_params['rep2']],
            [best_params['head']],
            best_params['drop'],
            best_params['ipm_weight']
        )
    elif model_name == "cevae":
        trainer = CEVAETrainer(
            X_trn.shape[1],
            best_params['latent_dim'],
            best_params['hidden_dim'],
            best_params['num_layers'],
            best_params['num_samples']
        )
    else:
        trainer = DragonNetTrainer(
            X_trn.shape[1],
            best_params['shared_hidden'],
            best_params['outcome_hidden']
        )

    trainer.fit(
        X_trn, t_trn, y_trn,
        X_test, t_test, y_test,
        lr=best_params['lr'],
        weight_decay=best_params['wd'],
        batch_size=best_params['bs'],
        epochs=best_params['epochs']
    )

    if model_name == "cevae":
        ite = trainer.predict(X_test)
    else:
        y0p, y1p = trainer.predict(X_test)
        ite = y1p - y0p

    return (ite, best_params) if provided_params is None else ite


## Fitting synthetic data generated by causl

### Data generation

In [ ]:
nrep = 20  # Number of repetitions
n_tr = 1000  # Training sample size
n_val = 400
n_te = 1000  # Testing sample size
strength_instr_values = np.arange(1,4.5,1)  # Varying strength of instrumental variables
nI = 5  # Fixed number of instrumental variables
nX = 5
nO = 0
nS = 0
binary_intervention = True
num_iters = 1000  # Fixed number of training iterations


# Initialize tracker for strength_instr
tracker = {strength_instr: {"fr": [], "dr": [], "causalegm":[], "tarnet":[], "cfrnet":[], "cevae":[], "dragonnet":[]}
           for strength_instr in strength_instr_values}
best_hps = {model: None for model in ["tarnet","cfrnet","cevae","dragonnet"]}
# Begin loop over strength_instr
for strength_instr in strength_instr_values:
    print(f"Running experiments for strength_instr = {strength_instr}")
    p = nI + nX + nO + nS  # Update the number of covariates
    
    for rep in tqdm(range(nrep)):
        # Generate training and testing data
        df_tr = generate_data_causl(n=n_tr, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)
        z_tr = torch.tensor(df_tr[[f"X{i}" for i in range(1, p + 1)]].values, dtype=torch.float32)
        x_tr = torch.tensor(df_tr['A'].values, dtype=torch.int32).view(-1, 1) if binary_intervention else \
            torch.tensor(df_tr['A'].values, dtype=torch.float32).view(-1, 1)
        y_tr = torch.tensor(df_tr['y'].values, dtype=torch.float32).view(-1, 1)
        
        z_tr_np = df_tr[[f"X{i}" for i in range(1, p + 1)]].values
        x_tr_np = df_tr['A'].values
        y_tr_np = df_tr['y'].values

        df_val = generate_data_causl(n=n_val, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)


        z_val_np = df_val[[f"X{i}" for i in range(1, p + 1)]].values
        x_val_np = df_val['A'].values
        y_val_np = df_val['y'].values

        df_te = generate_data_causl(n=n_te, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)

        z_te_np = df_te[[f"X{i}" for i in range(1, p + 1)]].values
        x_te_np = df_te['A'].values
        y_te_np = df_te['y'].values
        z_te = torch.tensor(z_te_np, dtype=torch.float32)

        model = Frengression(x_dim = x_tr.shape[1], y_dim = 1, z_dim =z_tr.shape[1], 
                             noise_dim=1, num_layer=3, hidden_dim=100, 
                             device=device, x_binary=binary_intervention, z_binary_dims=0)

        # Train Frengression model
        model.train_y(x=x_tr,
                      z=z_tr, 
                      y=y_tr, 
                      num_iters=num_iters, lr=1e-4, print_every_iter=1000)

        # Sample model distributions
        P0 = model.sample_causal_margin(torch.tensor([0], dtype=torch.int32), sample_size=n_te).numpy().reshape(-1, 1)
        P1 = model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=n_te).numpy().reshape(-1, 1)
        ate_fr = np.mean(P1) - np.mean(P0)

        # DR Estimation
        ate_dr, _ = dr_ate(x_tr_np, y_tr_np, z_tr_np ,x_te_np, y_te_np, z_te_np)

        for model in ["tarnet","cfrnet","cevae","dragonnet"]:
            if rep == 0:
                ite, best_hps[model] = tune_and_eval(
                    model,
                    z_tr_np, x_tr_np, y_tr_np,
                    z_val_np, x_val_np, y_val_np,
                    z_te_np,  x_te_np,  y_te_np,
                    provided_params=None,
                    n_trials=20
                )
            else:
                ite = tune_and_eval(
                    model,
                    z_tr_np, x_tr_np, y_tr_np,
                    z_val_np, x_val_np, y_val_np,
                    z_te_np,  x_te_np,  y_te_np,
                    provided_params=best_hps[model]
                )
            tracker[strength_instr][model].append(ite.mean())


        cegm_params = {'dataset': 'Semi_acic', 
                        'output_dir': '.', 
                        'v_dim': z_tr.shape[1], 
                        'z_dims': [1, 1, 1, 1], 
                        'lr': 0.0002, 
                        'alpha': 1, 
                        'beta': 1, 
                        'gamma': 10, 
                        'g_d_freq': 5, 
                        'g_units': [64, 64, 64, 64, 64], 
                        'e_units': [64, 64, 64, 64, 64], 
                        'f_units': [64, 32, 8], 
                        'h_units': [64, 32, 8], 
                        'dz_units': [64, 32, 8], 
                        'dv_units': [64, 32, 8], 'save_res': False, 'save_model': False, 'binary_treatment': True, 'use_z_rec': True, 'use_v_gan': True}
        egm_model = cegm.CausalEGM(params=cegm_params, random_seed=42)
        egm_model.train(data=[x_tr,y_tr,z_tr],n_iter=1000, verbose=False)
        ate_causalegm=egm_model.getCATE(z_te).mean()

        # Log results
        tracker[strength_instr]["fr"].append(ate_fr)
        tracker[strength_instr]["dr"].append(ate_dr)
        tracker[strength_instr]["dragonnet"].append(ate_dragonnet)
        


Running experiments for strength_instr = 1.0


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1: loss 2.3599,	loss_y 1.5776, 1.5881, 0.0211,	loss_eta 0.7823, 0.8246, 0.0847
Epoch 1000: loss 0.8436,	loss_y 0.2887, 0.5893, 0.6013,	loss_eta 0.5549, 1.0947, 1.0795


[I 2025-04-25 16:59:06,164] A new study created in memory with name: tarnet_tune
[I 2025-04-25 16:59:07,385] Trial 0 finished with value: 0.7634808421134949 and parameters: {'lr': 0.000796091902759762, 'wd': 4.284693309486253e-05, 'bs': 32, 'epochs': 60, 'rep1': 71, 'rep2': 145, 'head': 62, 'drop': 0.21202347713835268}. Best is trial 0 with value: 0.7634808421134949.
[I 2025-04-25 16:59:09,229] Trial 1 finished with value: 0.8567997217178345 and parameters: {'lr': 2.6995605248386923e-05, 'wd': 6.229634054446615e-07, 'bs': 32, 'epochs': 81, 'rep1': 139, 'rep2': 133, 'head': 124, 'drop': 0.40267417574487335}. Best is trial 0 with value: 0.7634808421134949.
[I 2025-04-25 16:59:11,837] Trial 2 finished with value: 0.7723536491394043 and parameters: {'lr': 0.00017660951481455182, 'wd': 4.114800080408878e-07, 'bs': 64, 'epochs': 178, 'rep1': 69, 'rep2': 143, 'head': 164, 'drop': 0.07095156374083322}. Best is trial 0 with value: 0.7634808421134949.
[I 2025-04-25 16:59:14,671] Trial 3 finished

🔍 Best params for tarnet: {'lr': 0.0009149603089507892, 'wd': 8.780817546298247e-05, 'bs': 32, 'epochs': 54, 'rep1': 54, 'rep2': 88, 'head': 62, 'drop': 0.2466612097068253}


...

The average treatment effect (ATE) is 1.9252685
Epoch 1: loss 2.3751,	loss_y 1.5408, 1.5816, 0.0814,	loss_eta 0.8343, 0.8608, 0.0530
Epoch 1000: loss 0.8299,	loss_y 0.2918, 0.5758, 0.5680,	loss_eta 0.5381, 1.0888, 1.1015


INFO 	 Training with 11 minibatches per epoch
INFO 	 Evaluating 1 minibatches
2025-04-25 17:14:26.496820: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-25 17:14:26.497150: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant

The average treatment effect (ATE) is 1.7876357
Epoch 1: loss 2.2696,	loss_y 1.5175, 1.5444, 0.0539,	loss_eta 0.7521, 0.7841, 0.0641
Epoch 1000: loss 0.8801,	loss_y 0.2945, 0.5679, 0.5469,	loss_eta 0.5856, 1.1066, 1.0420


INFO 	 Training with 11 minibatches per epoch
INFO 	 Evaluating 1 minibatches
2025-04-25 17:15:07.983906: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-25 17:15:07.984262: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant

The average treatment effect (ATE) is 1.8343469
Epoch 1: loss 2.2898,	loss_y 1.5228, 1.5384, 0.0313,	loss_eta 0.7670, 0.8040, 0.0740


In [72]:
egm_model.getCATE(z_tr).mean()

2025-04-25 15:02:38.511931: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-25 15:02:38.512307: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

1.8659396

In [25]:
np.mean(df_tr['y'][df_tr['A']==1])

2.400059767345967

In [47]:
model = Frengression(x_dim = x_tr.shape[1], y_dim = 1, z_dim =z_tr.shape[1], 
                             noise_dim=1, num_layer=3, hidden_dim=100, 
                             device=device, x_binary=binary_intervention, z_binary_dims=0)

In [51]:
model.train_y(x=x_tr,z=z_tr, 
                      y=y_tr, 
                      num_iters=1000, lr=1e-4, print_every_iter=1000)

Epoch 1: loss 1.8026,	loss_y 1.1675, 1.3866, 0.4383,	loss_eta 0.6351, 0.8774, 0.4847
Epoch 1000: loss 0.8402,	loss_y 0.2793, 0.5600, 0.5615,	loss_eta 0.5610, 1.1058, 1.0896


In [73]:
model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=n_p).mean()

AttributeError: module 'CausalEGM.model' has no attribute 'sample_causal_margin'

In [53]:
model.sample_causal_margin(torch.tensor([0], dtype=torch.int32), sample_size=n_p).mean()

tensor(0.0739)

In [38]:
y_tr.shape

torch.Size([1000, 1])